[![xeus-cling](images/xeus-cling.png)](https://github.com/jupyter-xeus/xeus-cling/)

A Jupyter kernel for C++ based on the `cling` C++ interpreter and the `xeus` native implementation of the Jupyter protocol, xeus.

- GitHub repository: https://github.com/jupyter-xeus/xeus-cling/
- Online documentation: https://xeus-cling.readthedocs.io/

## Usage

<div style="background: #efffed;
            border: 1px solid grey;
            margin: 8px 0 8px 0;
            text-align: center;
            padding: 8px; ">
    <i class="fa-play fa" 
       style="font-size: 40px;
              line-height: 40px;
              margin: 8px;
              color: #444;">
    </i>
    <div>
    To run the selected code cell, hit <pre style="background: #efffed">Shift + Enter</pre>
    </div>
</div>

## Output and error streams

`std::cout` and `std::cerr` are redirected to the notebook frontend.

In [1]:
#include <iostream>

std::cout << "some output" << std::endl;

some output


In [2]:
std::cerr << "some error" << std::endl;

some error


In [3]:
#include <stdexcept>

In [4]:
throw std::runtime_error("Unknown exception");

Standard Exception: Unknown exception

Omitting the `;` in the last statement of a cell results in an output being printed

In [5]:
int j = 5;

In [6]:
j

5

# Interpreting the C++ programming language

`cling` has a broad support of the features of C++. You can define functions, classes, templates, etc ...

## Functions

In [7]:
double sqr(double a)
{
    return a * a;
}

In [8]:
double a = 2.5;
double asqr = sqr(a);
asqr

6.2500000

## Classes

In [9]:
class Foo
{
public:

    virtual ~Foo() {}
    
    virtual void print(double value) const
    {
        std::cout << "Foo value = " << value << std::endl;
    }
};

In [10]:
Foo bar;
bar.print(1.2);

Foo value = 1.2


## Polymorphism

In [11]:
class Bar : public Foo
{
public:

    virtual ~Bar() {}
    
    virtual void print(double value) const
    {
        std::cout << "Bar value = " << 2 * value << std::endl;
    }
};

In [12]:
Foo* bar2 = new Bar;
bar2->print(1.2);
delete bar2;

Bar value = 2.4


## Templates

In [13]:
#include <typeinfo>

template <class T>
class FooT
{
public:
    
    explicit FooT(const T& t) : m_t(t) {}
    
    void print() const
    {
        std::cout << typeid(T).name() << " m_t = " << m_t << std::endl;
    }
    
private:
    
    T m_t;
};

In [14]:
FooT<double> foot(1.2);
foot.print();

d m_t = 1.2


## C++11 / C++14 support

In [15]:
class Foo11
{
public:
    
    Foo11() { std::cout << "Foo11 default constructor" << std::endl; }
    Foo11(const Foo11&) { std::cout << "Foo11 copy constructor" << std::endl; }
    Foo11(Foo11&&) { std::cout << "Foo11 move constructor" << std::endl; }
};

In [16]:
Foo11 f1;
Foo11 f2(f1);
Foo11 f3(std::move(f1));

Foo11 default constructor
Foo11 copy constructor
Foo11 move constructor


In [17]:
#include <vector>

std::vector<int> v = { 1, 2, 3};
auto iter = ++v.begin();
v

{ 1, 2, 3 }

In [18]:
*iter

2

... and also lambda, universal references, `decltype`, etc ...

## Documentation and completion

 - Documentation for types of the standard library is retrieved on cppreference.com.
 - The quick-help feature can also be enabled for user-defined types and third-party libraries. More documentation on this feature is available at https://xeus-cling.readthedocs.io/en/latest/inline_help.html.


In [19]:
?std::vector

No documentation found for std::vector


## Using the `display_data` mechanism

For a user-defined type `T`, the rich rendering in the notebook and JupyterLab can be enabled by by implementing the function `nl::json mime_bundle_repr(const T& im)`, which returns the JSON mime bundle for that type.

More documentation on the rich display system of Jupyter and Xeus-cling is available at https://xeus-cling.readthedocs.io/en/latest/rich_display.html

### Image example

In [20]:
#include <string>
#include <fstream>

#include "nlohmann/json.hpp"

#include "xtl/xbase64.hpp"

namespace nl = nlohmann;

namespace im
{
    struct image
    {   
        inline image(const std::string& filename)
        {
            std::ifstream fin(filename, std::ios::binary);   
            m_buffer << fin.rdbuf();
        }
        
        std::stringstream m_buffer;
    };
    
    nl::json mime_bundle_repr(const image& i)
    {
        auto bundle = nl::json::object();
        bundle["image/png"] = xtl::base64encode(i.m_buffer.str());
        return bundle;
    }
}

In file included from input_line_34:4:
In file included from /nix/store/pjfhjwrlnidj43bc36pyfdax4jxbrgk4-xtl/include/xtl/xbase64.hpp:16:
In file included from /nix/store/pjfhjwrlnidj43bc36pyfdax4jxbrgk4-xtl/include/xtl/xsequence.hpp:19:
In file included from /nix/store/pjfhjwrlnidj43bc36pyfdax4jxbrgk4-xtl/include/xtl/xmeta_utils.hpp:15:
In file included from /nix/store/pjfhjwrlnidj43bc36pyfdax4jxbrgk4-xtl/include/xtl/xfunctional.hpp:15:
/nix/store/pjfhjwrlnidj43bc36pyfdax4jxbrgk4-xtl/include/xtl/xtype_traits.hpp:46:49: error: no template named 'decay_t' in namespace 'std'; did you mean 'decay'?
        using type = decltype(std::declval<std::decay_t<T0>>() + std::declval<std::decay_t<T1>>());
                                           ~~~~~^
/nix/store/h31cy7jm6g7cfqbhc5pm4rf9c53i3qfb-gcc-9.3.0/lib/gcc/x86_64-unknown-linux-gnu/9.3.0/../../../../include/c++/9.3.0/type_traits:2010:11: note: 'decay' declared here
    class decay
          ^
In file included from input_line_34:4:
In file i

Interpreter Error: 

In [ ]:
im::image marie("images/marie.png");
marie

### Audio example

In [21]:
#include <string>
#include <fstream>

#include "nlohmann/json.hpp"

#include "xtl/xbase64.hpp"

namespace nl = nlohmann;

namespace au
{
    struct audio
    {   
        inline audio(const std::string& filename)
        {
            std::ifstream fin(filename, std::ios::binary);   
            m_buffer << fin.rdbuf();
        }
        
        std::stringstream m_buffer;
    };
    
    nl::json mime_bundle_repr(const audio& a)
    {
        auto bundle = nl::json::object();
        bundle["text/html"] =
           std::string("<audio controls=\"controls\"><source src=\"data:audio/wav;base64,")
           + xtl::base64encode(a.m_buffer.str()) +
            "\" type=\"audio/wav\" /></audio>";
        return bundle;
    }
}

In file included from input_line_35:4:
In file included from /nix/store/j87f699n8q6wrs4z553dzw1znydc8349-xtl/include/xtl/xbase64.hpp:17:
In file included from /nix/store/j87f699n8q6wrs4z553dzw1znydc8349-xtl/include/xtl/xsequence.hpp:20:
In file included from /nix/store/j87f699n8q6wrs4z553dzw1znydc8349-xtl/include/xtl/xmeta_utils.hpp:17:
In file included from /nix/store/j87f699n8q6wrs4z553dzw1znydc8349-xtl/include/xtl/xfunctional.hpp:16:
/nix/store/j87f699n8q6wrs4z553dzw1znydc8349-xtl/include/xtl/xtype_traits.hpp:88:49: error: no template named 'decay_t' in namespace 'std'; did you mean 'decay'?
        using type = decltype(std::declval<std::decay_t<T0>>() + std::declval<std::decay_t<T1>>());
                                           ~~~~~^
/nix/store/h31cy7jm6g7cfqbhc5pm4rf9c53i3qfb-gcc-9.3.0/lib/gcc/x86_64-unknown-linux-gnu/9.3.0/../../../../include/c++/9.3.0/type_traits:2010:11: note: 'decay' declared here
    class decay
          ^
In file included from input_line_35:4:
In file i

Interpreter Error: 

In [ ]:
au::audio drums("audio/audio.wav");
drums

### Display

In [22]:
#include "xcpp/xdisplay.hpp"

In file included from input_line_36:1:
/nix/store/kzddim5xn4j7h2z9n0rdv78hphc1y7ar-xeusCling/include/xcpp/xdisplay.hpp:25:9: error: use of undeclared identifier 'xeus'
        xeus::get_interpreter().display_data(
        ^
/nix/store/kzddim5xn4j7h2z9n0rdv78hphc1y7ar-xeusCling/include/xcpp/xdisplay.hpp:33:30: error: use of undeclared identifier 'xeus'
    void display(const T& t, xeus::xguid id, bool update=false)
                             ^
/nix/store/kzddim5xn4j7h2z9n0rdv78hphc1y7ar-xeusCling/include/xcpp/xdisplay.hpp:40:13: error: use of undeclared identifier 'xeus'
            xeus::get_interpreter().update_display_data(
            ^
/nix/store/kzddim5xn4j7h2z9n0rdv78hphc1y7ar-xeusCling/include/xcpp/xdisplay.hpp:48:13: error: use of undeclared identifier 'xeus'
            xeus::get_interpreter().display_data(
            ^
/nix/store/kzddim5xn4j7h2z9n0rdv78hphc1y7ar-xeusCling/include/xcpp/xdisplay.hpp:58:9: error: use of undeclared identifier 'xeus'
        xeus::get_interpret

Interpreter Error: 

In [23]:
xcpp::display(drums);

input_line_37:2:8: error: no type named 'display' in namespace 'xcpp'
 xcpp::display(drums);
 ~~~~~~^


Interpreter Error: 

### Update-display

In [24]:
#include <string>
#include "xcpp/xdisplay.hpp"

#include "nlohmann/json.hpp"

namespace nl = nlohmann;

namespace ht
{
    struct html
    {   
        inline html(const std::string& content)
        {
            m_content = content;
        }
        std::string m_content;
    };

    nl::json mime_bundle_repr(const html& a)
    {
        auto bundle = nl::json::object();
        bundle["text/html"] = a.m_content;
        return bundle;
    }
}

// A blue rectangle
ht::html rect(R"(
<div style='
    width: 90px;
    height: 50px;
    line-height: 50px;
    background-color: blue;
    color: white;
    text-align: center;'>
Original
</div>)");

In file included from input_line_38:2:
/nix/store/kzddim5xn4j7h2z9n0rdv78hphc1y7ar-xeusCling/include/xcpp/xdisplay.hpp:25:9: error: use of undeclared identifier 'xeus'
        xeus::get_interpreter().display_data(
        ^
/nix/store/kzddim5xn4j7h2z9n0rdv78hphc1y7ar-xeusCling/include/xcpp/xdisplay.hpp:33:30: error: use of undeclared identifier 'xeus'
    void display(const T& t, xeus::xguid id, bool update=false)
                             ^
/nix/store/kzddim5xn4j7h2z9n0rdv78hphc1y7ar-xeusCling/include/xcpp/xdisplay.hpp:40:13: error: use of undeclared identifier 'xeus'
            xeus::get_interpreter().update_display_data(
            ^
/nix/store/kzddim5xn4j7h2z9n0rdv78hphc1y7ar-xeusCling/include/xcpp/xdisplay.hpp:48:13: error: use of undeclared identifier 'xeus'
            xeus::get_interpreter().display_data(
            ^
/nix/store/kzddim5xn4j7h2z9n0rdv78hphc1y7ar-xeusCling/include/xcpp/xdisplay.hpp:58:9: error: use of undeclared identifier 'xeus'
        xeus::get_interpret

Interpreter Error: 

In [ ]:
xcpp::display(rect, "some_display_id");

In [ ]:
// Update the rectangle to be red
rect.m_content = R"(
<div style='
    width: 90px;
    height: 50px;
    line-height: 50px;
    background-color: red;
    color: white;
    text-align: center;'>
Updated
</div>)";

xcpp::display(rect, "some_display_id", true);

### Clear output

In [ ]:
#include <chrono>
#include <iostream>
#include <thread>

#include "xcpp/xdisplay.hpp"

In [ ]:
std::cout << "hello" << std::endl;
std::this_thread::sleep_for(std::chrono::seconds(1));
xcpp::clear_output();  // will flicker when replacing "hello" with "goodbye"
std::this_thread::sleep_for(std::chrono::seconds(1));
std::cout << "goodbye" << std::endl;

In [ ]:
std::cout << "hello" << std::endl;
std::this_thread::sleep_for(std::chrono::seconds(1));
xcpp::clear_output(true);  // prevents flickering
std::this_thread::sleep_for(std::chrono::seconds(1));
std::cout << "goodbye" << std::endl;

## Magics

Magics are special commands for the kernel that are not part of the C++ language.

They are defined with the symbol `%` for a line magic and `%%` for a cell magic.

More documentation for magics is available at https://xeus-cling.readthedocs.io/en/latest/magics.html.

In [22]:
#include <algorithm>
#include <vector>

In [23]:
std::vector<double> to_shuffle = {1, 2, 3, 4};

In [ ]:
%timeit std::random_shuffle(to_shuffle.begin(), to_shuffle.end());

[![xtensor](images/xtensor.png)](https://github.com/xtensor-stack/xtensor/)

- GitHub repository: https://github.com/xtensor-stack/xtensor/
- Online documentation: https://xtensor.readthedocs.io/
- NumPy to xtensor cheat sheet: http://xtensor.readthedocs.io/en/latest/numpy.html

`xtensor` is a C++ library for manipulating N-D arrays with an API very similar to that of numpy.

In [ ]:
#include <iostream>

#include "xtensor/xarray.hpp"
#include "xtensor/xio.hpp"
#include "xtensor/xview.hpp"

xt::xarray<double> arr1
  {{1.0, 2.0, 3.0},
   {2.0, 5.0, 7.0},
   {2.0, 5.0, 7.0}};

xt::xarray<double> arr2
  {5.0, 6.0, 7.0};

xt::view(arr1, 1) + arr2

Together with the C++ Jupyter kernel, `xtensor` offers a similar experience as `NumPy` in the Python Jupyter kernel, including broadcasting and universal functions.

In [21]:
#include <iostream>
#include "xtensor/xarray.hpp"
#include "xtensor/xio.hpp"

input_line_35:2:10: fatal error: 'xtensor/xarray.hpp' file not found
#include "xtensor/xarray.hpp"
         ^~~~~~~~~~~~~~~~~~~~


Interpreter Error: 

In [ ]:
xt::xarray<int> arr
  {1, 2, 3, 4, 5, 6, 7, 8, 9};

arr.reshape({3, 3});

std::cout << arr;

In [ ]:
#include "xtensor-blas/xlinalg.hpp"

In [ ]:
xt::xtensor<double, 2> m = {{1.5, 0.5}, {0.7, 1.0}};
std::cout << "Matrix rank: " << std::endl << xt::linalg::matrix_rank(m) << std::endl;
std::cout << "Matrix inverse: " << std::endl << xt::linalg::inv(m) << std::endl;
std::cout << "Eigen values: " << std::endl << xt::linalg::eigvals(m) << std::endl;

In [ ]:
xt::xarray<double> arg1 = xt::arange<double>(9);
xt::xarray<double> arg2 = xt::arange<double>(18);

arg1.reshape({3, 3});
arg2.reshape({2, 3, 3});

std::cout << xt::linalg::dot(arg1, arg2) << std::endl;